In [43]:
import cv2
import face_recognition
from torchvision import datasets, transforms, models
from PIL import Image,ImageDraw
import torch.nn as nn
import torch
from covid-mask-detector-master.covid-mask-detector.common.facedetector import FaceDetector
import matplotlib.pyplot as plt

SyntaxError: invalid syntax (<ipython-input-43-d980cfa27f0c>, line 7)

In [37]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Using Deep learning
## Model- resnet152, (No-mask working)

In [38]:
transform= transforms.Compose([transforms.ToPILImage(),transforms.RandomResizedCrop(224),transforms.ToTensor()])
model=models.resnet152(pretrained=True)
model.fc=nn.Sequential(nn.Linear(2048,2),
                      nn.ReLU(),nn.Dropout(0.2),
                      nn.LogSoftmax(dim=1))
model.load_state_dict(torch.load("D:\Stud\AI\dataset\CNN\mask_detect_weights_new.h5"))
model=model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [34]:
capture= cv2.VideoCapture(0)
face_encoding=[]
face_locations=[]
labels=["No-mask", "mask"]
color=[(10,0,255),(10,255,0)]
process_frame=True
recog = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
while True:
    ret,frame=capture.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray",gray)
    faces=recog.detectMultiScale(gray,1.1,5,minSize=(60,60),flags=cv2.CASCADE_SCALE_IMAGE)
    if len(faces)==0:
        cv2.putText(frame, "No face found",(30,30), font, 1.0, color[pred], 1)
    for (x,y,w,h) in faces:
        crop=frame[y:y+h,x:x+w]
        output=model(transform(crop).to(device).unsqueeze(0))
        _,pred=torch.max(output.data,1)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, labels[pred], (x, y-10), font, 1.0, color[pred], 1)
    cv2.imshow('Attendance Survillence', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

In [31]:
capture= cv2.VideoCapture(0)
face_encoding=[]
face_locations=[]
labels=["No_mask","Mask"]
color=[(10,0,255),(10,255,0)]
process_frame=True
while True:
    ret,frame=capture.read()
    small_frame=cv2.resize(frame,(0,0),fx=0.25, fy=0.25)
    rgb1=frame[:,:,::-1]
    rgb=small_frame[:,:,::-1]
    if process_frame:
        face_locations=face_recognition.face_locations(rgb)
        face_encoding = face_recognition.face_encodings(rgb, face_locations)
    process_frame = not process_frame
    for (top, right, bottom, left) in face_locations:
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        crop=frame[top:bottom,left:right]
        img=transform(crop).to(device)
        output=model(img.unsqueeze(0))
        _,pred=torch.max(output.data,1)
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, labels[pred], (30,30), font, 1.0, color[pred], 1)
    cv2.imshow('Attendance Survillence', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

# Using Harrcascade classifier

# Using Open Cv (working)


In [20]:
import cv2
import matplotlib.pyplot as plt
recog = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
mouth=cv2.CascadeClassifier('C:/Users/Ramprakash PG/anaconda3/envs/myenv/Lib/site-packages/cv2/data/haarcascade_mcs_mouth.xml')
print(mouth)

<CascadeClassifier 00000236569029B0>


In [29]:
threshold=80
video=cv2.VideoCapture(0)
color=[(10,0,255),(10,255,0)]
labels=["Mask","No_Mask"]
font = cv2.FONT_HERSHEY_DUPLEX

while True:
    ret, frame=video.read()
    frame=cv2.flip(frame,1)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    th,b_w=cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    face_detect=recog.detectMultiScale(gray,1.1,4)
    black_white=recog.detectMultiScale(b_w,1.1,4)
    cv2.imshow("b_w",b_w)
    if((len(face_detect)==0)and (len(black_white)==0)):
        cv2.putText(frame,"No face found",(30,30),font,1.0,color[0],2)
    elif((len(face_detect)==0) and len(black_white==1)):
        print("inside elif",len(black_white))
        print(face_detect,black_white)
        for (x,y,w,h) in face_detect:
            cv2.rectangle(frame,(x,y),(x+w,y+h),color[0],2)
            cv2.putText(frame,"Mask_found(White)",(30,30),font,1.0,color[1],2)
    else:
        for (x,y,w,h) in face_detect:
            cv2.rectangle(frame,(x,y),(x+w,y+h),color[1],2)
            crop=frame[x:y,x+y:y+h]
            roi_gray=gray[y:y+h,x:x+w]
            roi_color=frame[y:y+h,x:x+w]
            mouth_detect=mouth.detectMultiScale(roi_gray,1.5,5)
            if(len(mouth_detect)==0):
                cv2.putText(frame,labels[0],(30,30),font,1.0,color[1],2)
    cv2.imshow('Mask Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()    
    

inside elif 1
() [[139 149 270 270]]
inside elif 1
() [[137 148 265 265]]


In [42]:
recog = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
video=cv2.VideoCapture(0)
color=[(10,0,255),(10,255,0)]
while True:
    ret, frame=video.read()
    frame=cv2.flip(frame,1)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face_detect=recog.detectMultiScale(gray,1.1,4)
    for (x,y,w,h) in face_detect:
            cv2.rectangle(frame,(x,y),(x+w,y+h),color[0],2)
    cv2.imshow('Mask Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()    
